<a href="https://colab.research.google.com/github/ProfDoeg/Colegio_Invisible/blob/master/04_cuaderno.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Secure Communications

# Outline 

Apply asymmetric cyptography to build messaging protocol.

# Goals

- create and save keys
- download/upload keys
- decrypt spreadsheet access token
- understand `gspread`
- post name, public key, public message, and signature 
- sign message
- verify message
- write message to other participants
- write encrypted message to other participant
- write signed and encrypted message

# Preparing the Space

In [115]:
! ls

 Colegio_Invisible   llaves  'llaves (1).zip'   llaves.zip   sample_data


In [116]:
! pip install eciespy eth_keys gspread oauth2client
! git clone https://github.com/ProfDoeg/Colegio_Invisible.git
! pip install --upgrade gspread

fatal: destination path 'Colegio_Invisible' already exists and is not an empty directory.


# Make New Keys

In [ ]:
! mkdir llaves

In [ ]:
! python Colegio_Invisible/scripts/ecc_generate.py llaves/prvkey1.eck

Input password for encrypting keyfile: 
Repeat password for encrypting keyfile: 

Passwords match...
Private key generation complete
Private key encrypted and written to binary file: llaves/prvkey1.eck
50f98f74cffdba47767b7e80b89420810f44fdee0606bc37c5d7924dc758707d07cf18dc975d36a7d65cfcdd548cbe649f58dd977fd25aa162c7f2734eada0fb


In [ ]:
! python Colegio_Invisible/scripts/ecc_pubkey_extract.py llaves/prvkey1.eck llaves/pubkey1.eck 

Input password for private key file: 
Input password for public key file: 
Repeat password for public key file: 

Passwords match...
Completed public key extraction from: llaves/prvkey1.eck
Public key encrypted and written to binary file: llaves/pubkey1.eck
3941c6fff65d07f3fd40ab077c8ccb1a58f315b33302e7a4f25609543c82aa64e6bcc86d04bc15706ce7ff8e99fed1f329e3a7c99a0aa45d735885a8f4bcf47e55c640e4a8769f2f6273d771e79bed0c02ff54cf0511edf8face9d2f08cf701b


# Saving Keys

In [ ]:
! zip -r llaves.zip llaves

  adding: llaves/ (stored 0%)
  adding: llaves/statement1.txt (stored 0%)
  adding: llaves/pubkey1.eck (stored 0%)
  adding: llaves/sig1.sig (stored 0%)
  adding: llaves/prvkey1.eck (stored 0%)


In [ ]:
from google.colab import files
f=files.download('llaves.zip');

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

# Uploading Keys

In [ ]:
from google.colab import files
fu=files.upload();

Saving llaves.zip to llaves (1).zip


In [ ]:
! unzip llaves.zip

Archive:  llaves.zip
replace llaves/statement1.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
 extracting: llaves/statement1.txt   
replace llaves/pubkey1.eck? [y]es, [n]o, [A]ll, [N]one, [r]ename: a
error:  invalid response [a]
replace llaves/pubkey1.eck? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
 extracting: llaves/pubkey1.eck      
 extracting: llaves/sig1.sig         
 extracting: llaves/prvkey1.eck      


# Access Pubkey Using Script

In [119]:
ret=!{ 'python Colegio_Invisible/scripts/ecc_keydump.py llaves/pubkey1.eck'}
pubkey1=ret[-1]
pubkey1

'fb33714111767cf52bb4a4d7ed2d6caba5d2cbb4264b6cfe4797c2836a98376eb107207065f9f0b4ecc7b50bed3cee0628118931b648db60274268a88b40fd2d'

# `gspread` 


## Decrypt Credentials

Decrypt `client_secret.aes` using password

This will be communicated over voice by DrDoeg



In [ ]:
! python Colegio_Invisible/scripts/aes_decrypt.py Colegio_Invisible/secrets/client_secret.aes Colegio_Invisible/secrets/client_secret.json 

Input password for decryption: 
Success: Colegio_Invisible/secrets/client_secret.aes decryption complete
Written to: Colegio_Invisible/secrets/client_secret.json


`gspread` is a python module for programmatic interaction with Google Sheets. 

All the participants will have access to the same spreadsheet in the cloud.

https://docs.gspread.org/en/latest/user-guide.html

We use the decrypted credentials to authenticate and gain API access to the spreadsheet

In [120]:
import gspread
from oauth2client.service_account import ServiceAccountCredentials
# use creds to create a client to interact with the Google Drive API
scope = ['https://spreadsheets.google.com/feeds',
         'https://www.googleapis.com/auth/drive']
creds = ServiceAccountCredentials.from_json_keyfile_name('Colegio_Invisible/secrets/client_secret.json', scope)
client = gspread.authorize(creds)
sheets = client.open("signatures")

Here we can see the structure of the spreadsheet and all of the layers or `Worksheet`

In [121]:
sheets.worksheets()

[<Worksheet 'Multi' id:0>,
 <Worksheet 'Magic' id:2092809244>,
 <Worksheet 'ChorusSignature' id:549527758>,
 <Worksheet 'StealProposal' id:858983463>,
 <Worksheet 'StealSignature' id:475665302>,
 <Worksheet 'test' id:2104752728>]

Isolating the first sheet where we query the first row

In [122]:
sheet=sheets.worksheets()[0]
sheet.row_values(1)

['name', 'pubkey', 'statement', 'signature']

In [ ]:
#sheet.clear()
#sheet.update_cell(1, "name")
#sheet.append_row(["name", "pubkey","statement","signature"])
#sheet.__dir__()

On this layer we meet to post:

- `name` of participant
- `pubkey` public key of participant in hex
- `statement` is the declaration signed by participant 
- `signature` is the ecc signature of the statement

In [123]:
my_name=input('input name')
my_statement=input('input statement')
open('llaves/statement1.txt','wb').write(my_statement.encode())

input namefanny
input statementfanny 2 the moon


16

In addition we use the python scripts to perform the cryptography

In [124]:
! cat llaves/statement1.txt

fanny 2 the moon

In [125]:
ret = !{ 'python Colegio_Invisible/scripts/ecc_sign.py llaves/prvkey1.eck llaves/statement1.txt llaves/sig1.sig'}
sig1=ret[-1]
sig1

'94850a63f119038690e5a7c78f006439d48c7002282d57fbfb6cd63436a52dcd59acc5c2f8df82b3471ed5973e23f632d980a583c27c5ba1c8b684ca7bdc15c801'

Here we post to the Worksheet to create a roster of all the participants

In [126]:
sheet.append_row([my_name,pubkey1,my_statement,sig1])

{'spreadsheetId': '1z_iLwzlqa_JX7AdRR3OsBeFEgvS0jVYh1oo-X23bMOY',
 'tableRange': 'Multi!A1:D8',
 'updates': {'spreadsheetId': '1z_iLwzlqa_JX7AdRR3OsBeFEgvS0jVYh1oo-X23bMOY',
  'updatedCells': 4,
  'updatedColumns': 4,
  'updatedRange': 'Multi!A9:D9',
  'updatedRows': 1}}

This is a bird sound

In [ ]:
import IPython
IPython.display.Audio("Colegio_Invisible/sound/bird2.wav")

Reading the data a Pandas Dataframe

In [127]:
import pandas as pd

dataframe = pd.DataFrame(sheet.get_all_records())
dataframe

,name,pubkey,statement,signature
0,DrDoeg,a8a74228607881a4f3f80df52efdf0167fa327ce0d90ec...,I am Sirius,067aa773ee743b7c76be4c931a26fd6d2d7b5c583e8037...
1,Mono,a8a74228607881a4f3f80df52efdf0167fa327ce0d90ec...,Beneath the water,0774199a527b34cf07d88f908ccfec14755e9c80ffc481...
2,Homie,631b3ec130ab1af024059a45f3b43883f8379cc7ea0363...,i am a sweet homie,811e4a977631b5c9c3dc49ab1218f276a040ce585b79b3...
3,jojo,631b3ec130ab1af024059a45f3b43883f8379cc7ea0363...,jojo bee,2efa0cbb380007b941365ce0f97fc523746850f30ee215...
4,Captain,986e40e4d04689c17dd341f9fbd7f394200d729160570a...,ahoy,8fee10d21ecd3b95932b06f4ac346aa4df74951f4cf58a...
5,Ro,986e40e4d04689c17dd341f9fbd7f394200d729160570a...,Ro ro,54a16212d9af4a4f53947f6091a0137a292f53da836d8a...
6,Xdos,fb33714111767cf52bb4a4d7ed2d6caba5d2cbb4264b6c...,dang,d9460dc9b1c3bbad26a3c35b6bad5e04005f3e85bcee2e...
7,fanny,fb33714111767cf52bb4a4d7ed2d6caba5d2cbb4264b6c...,fanny 2 the moon,94850a63f119038690e5a7c78f006439d48c7002282d57...


We can verify statements. Here we use the `ecies` cryptography library directly without using the external scripts. 

In [128]:
import eth_keys
import ecies

eth_keys.datatypes.Signature(bytes.fromhex(sig1)).verify_msg(my_statement.encode(),eth_keys.keys.PublicKey(bytes.fromhex(pubkey1)))

True

In [188]:
def verify(sig,statement,pubkey):
  return eth_keys.datatypes.Signature(bytes.fromhex(sig)).verify_msg(statement.encode() if type(statement)!=bytes else statement,eth_keys.keys.PublicKey(bytes.fromhex(pubkey)))

In [189]:
verify(sig1,my_statement,pubkey1)

True

Here we verify all rows of the roster

In [190]:
dataframe.set_index('name').apply(lambda row: verify(row.signature,row.statement,row.pubkey),axis=1)

name
DrDoeg     True
Mono       True
Homie      True
jojo       True
Captain    True
Ro         True
Xdos       True
fanny      True
dtype: bool

Collecting the names of all the participants

In [132]:
participants=dataframe.name.tolist()
participants

['DrDoeg', 'Mono', 'Homie', 'jojo', 'Captain', 'Ro', 'Xdos', 'fanny']

Doing variable assignment for needed Worksheets

In [133]:
sheet1=sheets.worksheets()[1]
sheet2=sheets.worksheets()[2]
sheet3=sheets.worksheets()[3]
sheet1,sheet2,sheet3

(<Worksheet 'Magic' id:2092809244>,
 <Worksheet 'ChorusSignature' id:549527758>,
 <Worksheet 'StealProposal' id:858983463>)

In [145]:
#ONLY FOR DDOG MC
#df=pd.DataFrame('dada',columns=participants,index=participants)
#sheet1.update([df.columns.values.tolist()] + df.values.tolist())
#sheet2.update([df.columns.values.tolist()] + df.values.tolist())
#sheet3.update([df.columns.values.tolist()] + df.values.tolist())
#df

,DrDoeg,Mono,Homie,jojo,Captain,Ro,Xdos,fanny
DrDoeg,dada,dada,dada,dada,dada,dada,dada,dada
Mono,dada,dada,dada,dada,dada,dada,dada,dada
Homie,dada,dada,dada,dada,dada,dada,dada,dada
jojo,dada,dada,dada,dada,dada,dada,dada,dada
Captain,dada,dada,dada,dada,dada,dada,dada,dada
Ro,dada,dada,dada,dada,dada,dada,dada,dada
Xdos,dada,dada,dada,dada,dada,dada,dada,dada
fanny,dada,dada,dada,dada,dada,dada,dada,dada


## Worksheet 1

This is a place for casual public chats
- row is sender
- column is recipient

In [146]:
dataframe1 = pd.DataFrame(sheet1.get_all_records())
dataframe1.index=dataframe1.columns
dataframe1

,DrDoeg,Mono,Homie,jojo,Captain,Ro,Xdos,fanny
DrDoeg,dada,dada,dada,dada,dada,dada,dada,dada
Mono,dada,dada,dada,dada,dada,dada,dada,dada
Homie,dada,dada,dada,dada,dada,dada,dada,dada
jojo,dada,dada,dada,dada,dada,dada,dada,dada
Captain,dada,dada,dada,dada,dada,dada,dada,dada
Ro,dada,dada,dada,dada,dada,dada,dada,dada
Xdos,dada,dada,dada,dada,dada,dada,dada,dada
fanny,dada,dada,dada,dada,dada,dada,dada,dada


Make a little recipient widget interface

In [147]:
import ipywidgets as widgets
w=widgets.Dropdown(
    options=participants,
    value='DrDoeg',
    description='Recipient:',
    disabled=False,
)

In [148]:
#have a look
w

Dropdown(description='Recipient:', options=('DrDoeg', 'Mono', 'Homie', 'jojo', 'Captain', 'Ro', 'Xdos', 'fanny…

In [149]:
w.value

'fanny'

In [150]:
my_name

'fanny'

Creating messaging

In [151]:
p_dict=dict([ (e,i) for i,e in enumerate(participants)])
p_dict

{'Captain': 4,
 'DrDoeg': 0,
 'Homie': 2,
 'Mono': 1,
 'Ro': 5,
 'Xdos': 6,
 'fanny': 7,
 'jojo': 3}

Select your message recipient using the widget.

In [152]:
print(f'You are {my_name}')
print('Select a message recipient')
w

You are fanny
Select a message recipient


Dropdown(description='Recipient:', index=7, options=('DrDoeg', 'Mono', 'Homie', 'jojo', 'Captain', 'Ro', 'Xdos…

Write message 

Post to spreadsheet

refresh dataframe from worksheet

In [153]:
message=input('send a message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
sheet1.update_cell(*write_cell, message)
dataframe1 = pd.DataFrame(sheet1.get_all_records())
dataframe1.index=dataframe1.columns
dataframe1

send a messageyo fanny


,DrDoeg,Mono,Homie,jojo,Captain,Ro,Xdos,fanny
DrDoeg,dada,dada,dada,dada,dada,dada,dada,dada
Mono,dada,dada,dada,dada,dada,dada,dada,dada
Homie,dada,dada,dada,dada,dada,dada,dada,dada
jojo,dada,dada,dada,dada,dada,dada,dada,dada
Captain,dada,dada,dada,dada,dada,dada,dada,dada
Ro,dada,dada,dada,dada,dada,dada,dada,dada
Xdos,dada,dada,dada,dada,dada,dada,dada,dada
fanny,dada,dada,dada,dada,dada,dada,dada,yo fanny


# Encrypted Messaging

In [154]:
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

,DrDoeg,Mono,Homie,jojo,Captain,Ro,Xdos,fanny
DrDoeg,dada,dada,dada,dada,dada,dada,dada,dada
Mono,dada,dada,dada,dada,dada,dada,dada,dada
Homie,dada,dada,dada,dada,dada,dada,dada,dada
jojo,dada,dada,dada,dada,dada,dada,dada,dada
Captain,dada,dada,dada,dada,dada,dada,dada,dada
Ro,dada,dada,dada,dada,dada,dada,dada,dada
Xdos,dada,dada,dada,dada,dada,dada,dada,dada
fanny,dada,dada,dada,dada,dada,dada,dada,dada


In [155]:
def encrypt_message(hexPubKey,message):
    if type(message)!=bytes:
        message=message.encode()
    return ecies.encrypt(hexPubKey,message).hex()

In [156]:
w

Dropdown(description='Recipient:', index=7, options=('DrDoeg', 'Mono', 'Homie', 'jojo', 'Captain', 'Ro', 'Xdos…

In [157]:
encrypt_message(dataframe.set_index('name').pubkey[w.value],'hey guys hey hey')

'0463a91c178e1b195110265b0f3d5641cc54633db41ebd8a17533839ba3c857b3a066c92ba64630dc600aa79e534eb6f77f90e3bfab4d0bbea5c4d92c5afaaaa2877ad6ed62680147d7304cb723bf4b6c01aa170d8fe92c9cabb6d8ad28ecc2cbd7e0d510111a9277479078930ab2b6c12'

In [158]:
message=input('send an encrypted message')
write_cell=(p_dict[my_name]+2,p_dict[w.value]+1)
sheet2.update_cell(*write_cell, encrypt_message(dataframe.set_index('name').pubkey[w.value],message))
dataframe2 = pd.DataFrame(sheet2.get_all_records())
dataframe2.index=dataframe2.columns
dataframe2

send an encrypted messagehey hey


,DrDoeg,Mono,Homie,jojo,Captain,Ro,Xdos,fanny
DrDoeg,dada,dada,dada,dada,dada,dada,dada,dada
Mono,dada,dada,dada,dada,dada,dada,dada,dada
Homie,dada,dada,dada,dada,dada,dada,dada,dada
jojo,dada,dada,dada,dada,dada,dada,dada,dada
Captain,dada,dada,dada,dada,dada,dada,dada,dada
Ro,dada,dada,dada,dada,dada,dada,dada,dada
Xdos,dada,dada,dada,dada,dada,dada,dada,dada
fanny,dada,dada,dada,dada,dada,dada,dada,04ab9ca9ef0aea91234b6e36c2068b643a15a605a9bc5e...


In [159]:
def import_privKey(path,password):
  import hashlib
  if type(password)!=bytes:
    password=password.encode()
  f=open(path,'rb')
  decrypted_bytes=ecies.aes_decrypt(key=hashlib.sha256(password).digest(),cipher_text=f.read())
  privKey=eth_keys.keys.PrivateKey(decrypted_bytes)
  f.close()
  return privKey

def decrypt_message(privKey,cipherhex):
    ciphertext=bytes.fromhex(cipherhex)
    try:
      return ecies.decrypt(privKey.to_hex(),ciphertext)
    except:
      return None

In [160]:
decrypt_message(import_privKey('llaves/prvkey1.eck',''),dataframe2[my_name][my_name])


b'hey hey'

In [170]:
k=import_privKey('llaves/prvkey1.eck','')
tr=dataframe2[my_name].apply(lambda x: decrypt_message(k,x))
tr

DrDoeg           None
Mono             None
Homie            None
jojo             None
Captain          None
Ro               None
Xdos             None
fanny      b'hey hey'
Name: fanny, dtype: object

In [167]:
def sign_message(privKey,message):
    if type(message)!=bytes:
        message=message.encode()
    signature= privKey.sign_msg(message)
    return signature.to_bytes().hex()

In [181]:
mess=input('message')
mess_sig=sign_message(import_privKey('llaves/prvkey1.eck',''),mess)
mess_sig

messagehey guys


'e150cecc1e4eab64056825d45ca7ac00d18baaf9467f5a43c42e608339c502e23ac4fdf652b05f10f736eea2359c5163592a31f1194deeb61d1a3b2669e3e49d01'

In [182]:
emess=encrypt_message(pubkey1,mess)
emess

'0402c356c90fb7ec9da96fcb2c9e7a5fd785308864fbe11133025ca71355820a9669d2f5d90e5feeedf40dfc6373522ba1a889b9b8585696a0a1169f4ace4d2aa3eb7f80a96cc4a7c5946d0a91bf8ee359ec4dba96380363915c2c7b8e27a83705d7793ade86e01354'

In [183]:
emess_sig=sign_message(k,bytes.fromhex(emess))
emess_sig,len(emess_sig)

('0e1c62e889ee490e48a8ec78940e680b9ac0058206f516f03b5a318e969448d229d2e2cce6abe8949d027e7ad6357c4f7889cdbc8a55132dd4a60ebdc1ebe5ff00',
 130)

In [184]:
emess_sig+emess

'0e1c62e889ee490e48a8ec78940e680b9ac0058206f516f03b5a318e969448d229d2e2cce6abe8949d027e7ad6357c4f7889cdbc8a55132dd4a60ebdc1ebe5ff000402c356c90fb7ec9da96fcb2c9e7a5fd785308864fbe11133025ca71355820a9669d2f5d90e5feeedf40dfc6373522ba1a889b9b8585696a0a1169f4ace4d2aa3eb7f80a96cc4a7c5946d0a91bf8ee359ec4dba96380363915c2c7b8e27a83705d7793ade86e01354'

In [195]:
def decrypt_verify(pubkey,privkey,block):
  ver=verify(block[:130],bytes.fromhex(block[130:]),pubkey)
  return decrypt_message(privkey,block[130:]),ver

In [196]:
decrypt_verify(pubkey1,k,emess_sig+emess)

(b'hey guys', True)